# Preparing data for collaboration network
Here we only consider users v1 for size reasons.  
In this notebook, we try to create multiple collaboration graphs separated by time. The time is separated by month by looking up their commit overlapping time.